### Scrapping todays & yesturdays article on multiple pages of the African Examiner website by searching for the  country-name(or any keyword).

In [1]:
# importing modules
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import datetime
from datetime import date, timedelta, datetime
import time
import re

In [2]:
# Set up the web driver (provide the path to your Firefox web driver)
chrome_options = webdriver.FirefoxOptions()

chrome_options.add_argument("--window-size=1920x1080")
chrome_options.add_argument("--disable-notifications")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--verbose")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--disable-software-rasterizer")

driver = webdriver.Firefox(options=chrome_options)

driver.get("https://www.africanexaminer.com/")

In [3]:
# Get the input elements
search_input = driver.find_element(By.XPATH, "//input[@id='s']")
search_button = driver.find_element(By.XPATH, "//input[@id='searchsubmit']")

In [4]:
# Send the input to the webpage
search_input.send_keys('nigeria')
time.sleep(1)
search_button.click()

In [7]:
# Define today's date and yesterday's date
today = date.today()
yesterday = today - timedelta(days=1)

# This function will Check if the article date is within the desired date range
def post_date_check(post_date):
    # Extract the date
    date_match = re.search(r'(\w+),\s(\w+)\s(\d{1,2}(?:st|nd|rd|th)),\s(\d{4})', post_date)
    
    if date_match:
        day_name, month_name, day_with_suffix, year = date_match.groups()
        # Use a regular expression to extract the day without suffix
        day = int(re.search(r'\d+', day_with_suffix).group())
        month_map = {
            'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6,
            'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12
        }
        month = month_map.get(month_name)
        article_date = datetime(int(year), month, day).date()

        if article_date == today or article_date == yesterday:
            return True
        else:
            return False

In [23]:
# list for store the articles url
todays_and_yesturdays_article_urls = []

# Scrapping articles url from 'Latest-news' section
Latest_news = driver.find_elements(By.XPATH, "//ul[@class='mid-cat']//a")

for i in Latest_news:
    todays_and_yesturdays_article_urls.append(i.get_attribute("href"))


In [26]:
# Scrapping articles with pagination
new_articles = True

# Loop through the pages
while new_articles:
    
    # Scrolling down to load more search results
    for _ in range(5):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)
    
    # collecting all articles from the current page
    article_list = driver.find_elements(By.CLASS_NAME, "post")
    
    # Extract the URLs of today's and yesterday's articles 
    for article in article_list:
        # Extract the date
        date_text = article.find_element(By.CLASS_NAME, "postdate").text
        date_match = post_date_check(date_text)

        if date_match:
            a_link = article.find_element(By.TAG_NAME, "a").get_attribute("href")
            todays_and_yesturdays_article_urls.append(a_link)
        else:
            new_articles = False
            break
        
    # Find and click the pagination link for the next page
    try:
        next_page_link = driver.find_element(By.XPATH, "//div[@class='navigation']/a[contains(text(), 'Next Page')]")
        next_page_link.click()
    except NoSuchElementException:
        print("No 'Next Page' link found. Exiting the loop.")
        break  # Exit the loop if there is no "Next Page" link

In [43]:
len(todays_and_yesturdays_article_urls)

14

In [44]:
# Set pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# Initialize an empty DataFrame
df = pd.DataFrame(columns=["Headline", "Author", "Post_date", "Content"])

for article_url in todays_and_yesturdays_article_urls:
    driver.get(article_url)
    post_date = driver.find_element(By.CLASS_NAME, "postdate").text
    
    if post_date_check(post_date):
        # collecting the data
        headline = driver.find_element(By.XPATH, "//h1[@class='singlePageTitle']").text
        author = driver.find_element(By.CLASS_NAME, "postauthor").text
        p_content = driver.find_elements(By.XPATH, "//div[@class='post']//p")
        content = ''
        for i in p_content:
            content += i.text
            
        # Storing the date in to dataframe
        df = df.append({"Headline": headline, "Author": author, "Post_date": post_date, "Content": content}, ignore_index=True)


C:\Users\Admin\AppData\Local\Temp\ipykernel_4312\3963198717.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"Headline": headline, "Author": author, "Post_date": post_date, "Content": content}, ignore_index=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_4312\3963198717.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"Headline": headline, "Author": author, "Post_date": post_date, "Content": content}, ignore_index=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_4312\3963198717.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"Headline": headline, "Author": author, "Post_date": post_date, "Content": content}, ignore_index=True)
C:\Users\Admin\AppData\Local\Temp\

In [49]:
# removing duplicated article if any
df = df.drop_duplicates()

In [52]:
# df.sample(2)

In [ ]:
# storing data in to excel file
df.to_excel('African_Examiner_article.xlsx', index=False)

In [53]:
driver.quit()

# Rough work

In [28]:
# Testing code with single url
signle_article_url = todays_and_yesturdays_article_urls[0]

In [29]:
driver.get(signle_article_url)

In [30]:
headline = driver.find_element(By.XPATH, "//h1[@class='singlePageTitle']").text
headline

'[OPINION] Can INEC Deliver Hitch-Free Election In Kogi, Bayelsa, Imo?'

In [31]:
p_date = driver.find_element(By.CLASS_NAME, "postdate").text
p_date

'Tuesday, October 24th, 2023'

In [34]:
p_author = driver.find_element(By.CLASS_NAME, "postauthor").text
p_author

'Posted by African Examiner'

In [35]:
p_con = driver.find_elements(By.XPATH, "//div[@class='post']//p")
p_con_text = ''
for i in p_con:
    p_con_text += i.text

In [37]:
# p_con_text

In [42]:
if post_date_check('Saturday, October 24st, 2023'):
    print('T')
else:
    print('F')

T
